In [6]:
import pandas as pd

df = pd.read_csv('/workspaces/codespaces-blank/Top-100 Trending Books.csv')

print(df.head())
print(df.info())

   Rank                                         book_title  book_price  \
0     1                       Iron Flame (The Empyrean, 2)       18.42   
1     2                                    The Woman in Me       20.93   
2     3                                  My Name Is Barbra       31.50   
3     4  Friends, Lovers, and the Big Terrible Thing: A...       23.99   
4     5                              How to Catch a Turkey        5.65   

   rating            author  year_of_publication               genre  \
0     4.1    Rebecca Yarros                 2023     Fantasy Romance   
1     4.5    Britney Spears                 2023              Memoir   
2     4.5  Barbra Streisand                 2023       Autobiography   
3     4.4     Matthew Perry                 2023              Memoir   
4     4.8      Adam Wallace                 2018  Childrens, Fiction   

                                                 url  
0  amazon.com/Iron-Flame-Empyrean-Rebecca-Yarros/...  
1  amazon.co

In [7]:
df = df.dropna()
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 97 entries, 0 to 99
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Rank                 97 non-null     int64  
 1   book_title           97 non-null     object 
 2   book_price           97 non-null     float64
 3   rating               97 non-null     float64
 4   author               97 non-null     object 
 5   year_of_publication  97 non-null     int64  
 6   genre                97 non-null     object 
 7   url                  97 non-null     object 
dtypes: float64(2), int64(2), object(4)
memory usage: 6.8+ KB
None


In [8]:
from transformers import pipeline

# Initialize the question-answering pipeline
qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")


In [9]:
def get_book_recommendations(query, context):
    # Use the question-answering pipeline to process the query
    response = qa_pipeline(question=query, context=context)
    return response


In [11]:
# Generate a context string from your dataset
def generate_context(df):
    context = ""
    for index, row in df.iterrows():
        context += f"Title: {row['book_title']}, Author: {row['author']}, Genre: {row['genre']}\n"
    return context

context = generate_context(df)


In [12]:
from fastapi import FastAPI, Query

app = FastAPI()

@app.get("/ask_book_recommendation")
def ask_book_recommendation(query: str = Query(..., description="Ask about top books")):
    context = generate_context(df)
    answer = get_book_recommendations(query, context)
    return answer
